Saving data from yfinance till date

In [ ]:
# import pandas as pd
# import yfinance as yf

In [111]:
# df = pd.read_csv("../data/bitcoin_price.csv")
# df.head()


In [112]:
# import yfinance as yf
# import pandas as pd
# import datetime as dt

# today = dt.datetime.today()
# two_weeks = dt.timedelta(weeks=2)
# cutoff = today - two_weeks
# start_date = dt.datetime(2014, 9, 17)

# btc_data = yf.download("BTC-USD", start=start_date, end=today)

# print("Raw latest index date:", btc_data.index.max())

# btc_data = btc_data.reset_index()

# print("After reset_index() — latest 'Date':", btc_data['Date'].max())

# btc_data['Date'] = pd.to_datetime(btc_data['Date'])

# btc_data['Adj Close'] = btc_data['Close']

# btc_data = btc_data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# train_data = btc_data[btc_data['Date'] < cutoff]
# test_data = btc_data[btc_data['Date'] >= cutoff]

# print("Train:", train_data['Date'].min().date(), "to", train_data['Date'].max().date())
# print("Test:", test_data['Date'].min().date(), "to", test_data['Date'].max().date())


In [ ]:
# train_data.to_csv("../data/raw/train.csv", index = False)
# test_data.to_csv("../data/raw/test.csv", index = False)

In [ ]:
# train = pd.read_csv("../data/raw/train.csv")
# test = pd.read_csv("../data/raw/test.csv")

In [ ]:
# train.reset_index()

In [113]:
# test.reset_index()

In [114]:
# train = train.drop(index=0).reset_index(drop=True)
# test = test.drop(index=0).reset_index(drop=True)
# train.tail()

In [107]:
# test.to_csv("../data/processed/test.csv", index=False)
# train.to_csv("../data/processed/train.csv", index = False)

In [210]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df = pd.read_csv("../data/processed/train.csv")

In [121]:
df['Date'] = pd.to_datetime(df['Date'], format = "%Y-%m-%d")

In [125]:
df.set_index('Date', inplace=True)

In [ ]:
df.head()

In [ ]:
df.loc['2021']

In [ ]:
#Resampling to monthly freq and mean price
df.resample('M').mean()

In [146]:
#7 - Day rolling avg of 'Close'
df['df_7dayrolling'] = df['Close'].rolling(window=7).mean()

In [ ]:
df[['Close', 'df_7dayrolling']].loc['2023'].plot()
plt.show()

In [150]:
#Find out the highest avg month
df.resample('M').mean()['Close'].idxmax()

Timestamp('2025-06-30 00:00:00')

In [156]:
df.drop('df_7day',axis = 1, inplace = True)

In [157]:
#Calculate Daily returns
df['daily_returns'] = df['Close'].pct_change()*100

In [ ]:
#Days with more than 10% change in Closing Price
df[abs(df['daily_returns']) > 10].head() 

In [ ]:
#Some Data vizualization
#Daily closing price
df['Close'].plot(title = 'Daily closing price', figsize=(10,4))
plt.show()

In [ ]:
#Plot the yearly volume
df.resample('Y').sum()['Volume'].plot(figsize=(10,4), title='Yearly Volume')
plt.show()

In [ ]:
#Plot Closing price and 30-day rolling Volume
df['30-days-rolling-volume'] = df['Volume'].rolling(window=30).mean()
df['30-days-rolling-volume'].plot(figsize=(10,4), title='30 Days Rolling Volume', legend=True)
ax = df['Close'].plot(secondary_y = True, legend=True)
ax.set_ylabel('Volume')
plt.show()

In [ ]:
#Correlation between Close and 30-days-rolling-volume
df[['Close', '30-days-rolling-volume']].corr()

Data Manipulation

In [ ]:
#Checking missing values
df.isnull().sum()

#Filling them
df['30-days-rolling-volume'] =df['30-days-rolling-volume'].bfill()
df['df_7dayrolling'] =df['df_7dayrolling'].bfill() 
df['daily_returns'] =df['daily_returns'].bfill()



In [ ]:
df.isnull().sum()


In [190]:
#Extracting time variable
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.day_name()
df['weekday numeric'] = df.index.weekday
df['is_weekend'] = df.index.weekday > 4

In [192]:
#Lagged values
df['closed_lag1'] = df['Close'].shift(1)
df['closed_lag2'] = df['Close'].shift(2)

Seasonality


In [ ]:
#month plot - seasonality
monthly_avg = df['Close'].resample('M').mean()
month_plot(monthly_avg, ylabel='Closing')

In [199]:
# #quater plot - seasonality
# quarter_plot = df['Close'].resample('Q').mean()
# quarter_plot(monthly_avg, ylabel='Closing')

In [ ]:
#Seasonal decomposition - yearly
decomposition = seasonal_decompose(df['Close'],
                                    model= 'mul',
                                    period = 365)

fig = decomposition.plot()
fig.set_size_inches(10,8)
plt.show()

In [ ]:
#Auto-Correlation plots (ACF)
fig, ax = plt.subplots(figsize = (10,4))
plot_acf(df['Close'], lags = 100, ax=ax)
plt.show()

In [ ]:
#Partial-Auto correlation
fig, ax = plt.subplots(figsize = (10,4))
plot_pacf(df['Close'], lags = 100, ax=ax)
plt.show()